In [1]:
import sys
sys.path.append('..')

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
from gerumo.data.generators import MonoGenerator
from gerumo.data.dataset import (
    load_dataset, aggregate_dataset
)
from gerumo.data.constants import (
    TELESCOPES
)
from gerumo.utils.structures import (
    Telescope, Event, Observations,
    Task, ReconstructionMode
)
from gerumo.data.mappers import (
    OnevsAllClassification,
    SimpleCategorical, SimpleRegression, SimpleSquareImage
)
from gerumo.visualization.explore import plot_telescope_geometry
%matplotlib inline

In [4]:
events_filepath = "/home/asuka/projects/gerumo2/tools/output/splits/train_events.parquet"
telescopes_filepath = "/home/asuka/projects/gerumo2/tools/output/splits/train_telescopes.parquet"

In [5]:
dataset = load_dataset(events_filepath, telescopes_filepath)
dataset = aggregate_dataset(dataset)

# Telescopes

In [6]:
lst = Telescope("LST","LST","LSTCam")
mst = Telescope("MST","MST","FlashCam")
sst = Telescope("ASTRI","SST","CHEC")
#TELESCOPES

In [7]:
load_from_cache = True

In [ ]:
for telescope in (lst, mst, sst):
    if load_from_cache:
        pispox = telescope.get_pixels_positions()
    else:
        pispox = telescope.get_pixels_positions(hdf5_filepath=dataset.loc[0, "hdf5_filepath"])
    plot_telescope_geometry(telescope, pispox)

In [ ]:
for telescope in (lst, mst, sst):
    if load_from_cache:
        pispox, _ = telescope.get_aligned_pixels_positions()
    else:
        pispox, _ = telescope.get_aligned_pixels_positions(hdf5_filepath=dataset.loc[0, "hdf5_filepath"])
    plot_telescope_geometry(telescope, pispox, is_aligned=False)


In [ ]:
for telescope in (lst, mst, sst):
    telescope.save_geometry_to_cache()

# Image mappers


In [7]:
import numpy as np
import matplotlib.pyplot as plt
IMAGES_SIZE = {
    "lst" : (55, 47, 1),
    "mst": (84, 29, 1),
    "sst": (72, 25, 1)
}

In [8]:
telescope = mst

i = 1
x, y = telescope.get_aligned_pixels_positions()[i] 
w,h = telescope.get_aligned_pixels_positions()[i].max(axis=1) + 1
c = 1

In [ ]:
image_size = (h,w,c)
canvas = np.zeros(image_size, dtype="float32")
canvas[y, x, 0] = 1

plt.imshow(canvas)

# Generators

## Regression

In [21]:
generator = MonoGenerator(
    dataset, sst, 4,
    SimpleSquareImage(image_channels=["image", "peak_time", "image_mask"], telescope_features=[], mode=ReconstructionMode.SINGLE),
    SimpleRegression(targets=["true_az", "true_alt"], domains=[(-2,2),(-2,2)]))

In [22]:
generator[0][0]

In [ ]:
generator[0][1]

## Catagorical

In [ ]:
generator = MonoGenerator(
    dataset, Telescope("MST","MST","FlashCam"), 4,
    SimpleSquareImage(image_channels=["image", "image_mask"], telescope_features=[], mode=ReconstructionMode.SINGLE),
    SimpleCategorical(target="particle_type", num_classes=2, classes=["proton", "gamma"])
)

In [ ]:
generator[0][0]

In [ ]:
generator[0][1]

## One vs all

In [ ]:
generator = MonoGenerator(
    dataset, Telescope("MST","MST","FlashCam"), 4,
    SimpleSquareImage(image_channels=["image", "image_mask"], telescope_features=[], mode=ReconstructionMode.SINGLE),
    OnevsAllClassification(target="particle_type", num_classes=2, classes=["gamma"])
)

In [ ]:
generator[0][0]

In [ ]:
generator[0][1]